In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from scipy.stats import norm, ttest_ind
from datetime import datetime
import os

# Задание 1
Раньше мы оценивали эксперимент "Refactoring backend", предполагая, что данные времени работы бэкенда независимые. Теперь мы можем корректно оценить этот эксперимент, даже если данные зависели бы от пользователей.

Оцените эксперимент "Refactoring backend" с использованием линеаризации в предположении, что данные пользователей зависимы.

Данные эксперимента "Refactoring backend": 2022-04-13/2022-04-13T12_df_web_logs.csv и 2022-04-13/experiment_users.csv. Эксперимент проводился с 2022-04-05 по 2022-04-12. Метрика — среднее время обработки запроса.

В качестве ответа введите p-value с точность до 4-го знака после точки.

In [60]:
web_logs['date'].min()

Timestamp('2022-02-03 23:45:37')

In [61]:
web_logs['date'].max()

Timestamp('2022-04-13 11:59:59')

In [39]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-13/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

web_logs = read_database('2022-04-13T12_df_web_logs.csv')
experiment_users = read_database('experiment_users.csv')

web_logs['date'] = pd.to_datetime(web_logs['date'])

In [64]:
web_logs

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8
...,...,...,...,...
2401704,f721f6,b,2022-04-13 11:59:54,78.3
2401705,1238da,b,2022-04-13 11:59:56,62.3
2401706,cc7e71,m,2022-04-13 11:59:58,67.4
2401707,1587e4,b,2022-04-13 11:59:59,74.3


In [77]:
web_logs[(web_logs['date'] >= datetime(2022, 4, 5)) & (web_logs['date'] <= datetime(2022, 4, 12))].groupby('user_id', as_index=False)['load_time'].sum().sort_values(['user_id'])

,user_id,load_time
0,0000de,711.2
1,0000e7,548.0
2,0001ff,432.6
3,0006bb,305.1
4,000ea9,745.1
...,...,...
35640,fff7ff,569.2
35641,fff8bf,667.0
35642,fff960,402.9
35643,fff98d,459.2


In [80]:
df_experiment

,user_id,x,y
0,0000de,711.2,10
1,0000e7,548.0,8
2,0001ff,432.6,6
3,0006bb,305.1,4
4,000ea9,745.1,10
...,...,...,...
35640,fff7ff,569.2,8
35641,fff8bf,667.0,9
35642,fff960,402.9,6
35643,fff98d,459.2,7


In [84]:
test = web_logs[(web_logs['date'] >= datetime(2022, 4, 5)) & (web_logs['date'] <= datetime(2022, 4, 12))]

In [88]:
test[test[['user_id', 'date']].duplicated(keep=False)].sort_values(['user_id', 'date'])

,user_id,page,date,load_time
2169797,005838,m,2022-04-06 18:53:26,73.3
2169798,005838,m,2022-04-06 18:53:26,79.1
2131187,008aee,b,2022-04-05 16:40:44,68.4
2131188,008aee,m,2022-04-05 16:40:44,98.7
2279369,021cd6,m,2022-04-09 17:24:54,53.6
...,...,...,...,...
2331484,ff482e,m,2022-04-10 21:53:52,49.1
2344924,ff5447,b,2022-04-11 14:44:29,80.6
2344925,ff5447,m,2022-04-11 14:44:29,62.2
2285357,ffec21,b,2022-04-09 19:07:09,79.9


In [79]:
df_experiment = (
    web_logs[(web_logs['date'] >= datetime(2022, 4, 5)) & (web_logs['date'] <= datetime(2022, 4, 12))]
    .groupby('user_id').agg(x=('load_time', 'sum'), y=('date', 'count'))
    .reset_index()
).copy()

exp_data = experiment_users.merge(df_experiment, how='left', on='user_id')

In [81]:
control = exp_data[exp_data['pilot'] == 0]
pilot = exp_data[exp_data['pilot'] == 1]

coeff_lin = np.sum(control['x']) / np.sum(control['y'])

control['lin_metric'] = control['x'] - coeff_lin * control['y']
pilot['lin_metric'] = pilot['x'] - coeff_lin * pilot['y']

_, p_value = stats.ttest_ind(control['lin_metric'], pilot['lin_metric'])

/var/folders/1p/ybd4mcmj71j_w9gphd0xdrgc6vsdmk/T/ipykernel_38029/2457346965.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['lin_metric'] = control['x'] - coeff_lin * control['y']
/var/folders/1p/ybd4mcmj71j_w9gphd0xdrgc6vsdmk/T/ipykernel_38029/2457346965.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pilot['lin_metric'] = pilot['x'] - coeff_lin * pilot['y']


In [82]:
print(f'p_value = {round(p_value, 4)}')

p_value = 0.0442


In [83]:
p_value

0.04416800272407419